In [ ]:
import requests
import pandas as pd

# Read API key from .txt file
with open('../../Papers/mode choice/TRB2024/HHTS2012/codes_biogeme/API.txt', 'r') as f:
    api_key = f.read().strip()

#### OD Matrix creation

In [ ]:
# Read location data from CSV
data1=pd.read_excel('ODcostmatrix.xlsx',sheet_name ='ODcostmatrix',index_col=0)
data1

In [ ]:
base_url = "http://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?"

locations = ";".join([f"{lat},{long}" for lat, long in zip(data1['lat'], data1['long'])])

url = f"{base_url}origins={locations}&destinations={locations}&travelMode=driving&key={api_key}"

response = requests.get(url)
result = response.json()

distance_matrix = result['resourceSets'][0]['resources'][0]['results']

cost_matrix = pd.DataFrame(index=data1.index, columns=data1.index)

for i in range(len(data1)):
    for j in range(len(data1)):
        cost_matrix.iloc[i, j] = distance_matrix[i * len(data1) + j]['travelDistance']/1.6

cost_matrix

### Mode Specific Travel time in a single Row

In [ ]:
# # Read location data from CSV
# data2=pd.read_excel('ODcostmatrix.xlsx',sheet_name ='self_made_data',index_col=0)
# data2.head()

In [ ]:
# # Function to get trip time for a specific mode
# def get_trip_time(origin_lat, origin_long, dest_lat, dest_long, mode_choice, departure_time=None):
#     base_url = 'https://dev.virtualearth.net/REST/V1/Routes'
#     params = {
#         'wp.0': f'{origin_lat},{origin_long}',
#         'wp.1': f'{dest_lat},{dest_long}',
#         'key': api_key,
#     }

#     if mode_choice == 'transit':
#         base_url += '/Transit'
#         if departure_time is not None:
#             params['dateTime'] = departure_time
#             params['timeType'] = 'Departure'

#     elif mode_choice == 'walking':
#         base_url += '/Walking'

#     elif mode_choice == 'bike':
#         base_url += '/Walking'

#     else:
#         base_url += '/Driving'

#     response = requests.get(base_url, params=params)
#     data = response.json()

#     # Extract the trip duration from the response
#     if mode_choice == 'transit':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

#     elif mode_choice == 'walking':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']
    
#     elif mode_choice=='bike':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

#     else:
#         trip_duration = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['travelDuration']

#     return trip_duration

In [ ]:
# # Iterate through each row and get trip time for each mode
# for index, row in data2.iterrows():
#     origin_lat = row['o_lat']
#     origin_long = row['o_long']
#     dest_lat = row['d_lat']
#     dest_long = row['d_long']
#     mode_choice = row['mode_choice']

#     if mode_choice == 'transit':
#     # Specify the departure_time for transit mode (replace '2023-07-20T08:00:00' with your desired time)
#         departure_time = '2023-07-20T08:00:00'
#         transit_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'transit', departure_time)
#         travel_time = transit_time / 60
    
#     elif mode_choice == 'walking':
#         walk_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'walking')
#         travel_time = walk_time / 60
    
#     elif mode_choice == 'bike':
#         bike_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'bike')
#         travel_time = bike_time / (3 * 60)
    
#     else:
#         car_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'car')
#         travel_time = car_time / 60
    
#     data2.at[index, 'travel_time'] = travel_time

# # Print the results
# print(data2)

### OD matrix CVTD Blue loop

In [ ]:
# Read location data from CSV
data2=pd.read_excel('ODcostmatrix.xlsx',sheet_name ='CVTD_blueloop_stop',index_col=0)
data2.head()

In [ ]:
# Split the 'x,y' column into 'Latitude' and 'Longitude' columns
data2[['o_lat', 'o_long']] = data2['originx,y'].str.split(',', expand=True)
data2[['d_lat', 'd_long']] = data2['destx,y'].str.split(',', expand=True)
data2.head()

In [ ]:
columns_to_drop = ['originx,y', 'destx,y']
data2.drop(columns_to_drop, axis=1, inplace=True)
data2.head()

OD Matrix

In [ ]:
# base_url = "http://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?"

# locations = ";".join([f"{lat},{long}" for lat, long in zip(data2['o_lat'], data2['o_long'])])

# url = f"{base_url}origins={locations}&destinations={locations}&travelMode=driving&key={api_key}"

# response = requests.get(url)
# result = response.json()

# distance_matrix = result['resourceSets'][0]['resources'][0]['results']

# cost_matrix = pd.DataFrame(index=data2.index, columns=data2.index)

# for i in range(len(data2)):
#     for j in range(len(data2)):
#         cost_matrix.iloc[i, j] = distance_matrix[i * len(data2) + j]['travelDistance']/1.6   # Distance in miles

# cost_matrix      # Bing Map API has 25*25 OD matrix limitation

#### Travel time for each mode choices for each choice sets

In [ ]:
# # Travel time for each mode choices for each choice sets
# def get_trip_time(origin_lat, origin_long, dest_lat, dest_long, mode_choice, departure_time=None):
#     base_url = 'https://dev.virtualearth.net/REST/V1/Routes'
#     params = {
#         'wp.0': f'{origin_lat},{origin_long}',
#         'wp.1': f'{dest_lat},{dest_long}',
#         'key': api_key,
#     }

#     if mode_choice == 'transit':
#         base_url += '/Transit'
#         if departure_time is not None:
#             params['dateTime'] = departure_time
#             params['timeType'] = 'Departure'

#     elif mode_choice == 'walking':
#         base_url += '/Walking'

#     elif mode_choice == 'carpool':
#         base_url += '/Driving'

#     else:
#         base_url += '/Driving'

#     response = requests.get(base_url, params=params)
#     data = response.json()

#     # Extract the trip duration from the response
#     if mode_choice == 'transit':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

#     elif mode_choice == 'walking':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

#     else:
#         trip_duration = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['travelDuration']

#     return trip_duration



# # Add new columns for trip times
# data2['car_time'] = None
# data2['carpool_time'] = None
# data2['transit_time'] = None
# data2['walk_time'] = None

# # Iterate through each row and get trip time for each mode
# for index, row in data2.iterrows():
#     origin_lat = row['o_lat']
#     origin_long = row['o_long']
#     dest_lat = row['d_lat']
#     dest_long = row['d_long']

#     car_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'car')
#     data2.at[index, 'car_time'] = car_time / 60

#     carpool_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'carpool')
#     data2.at[index, 'carpool_time'] = carpool_time * 1.20 / 60

#     # Specify the departure_time for transit mode (replace '2023-07-20T08:00:00' with your desired time)
#     departure_time = '2023-07-20T08:00:00'
#     transit_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'transit', departure_time)
#     data2.at[index, 'transit_time'] = transit_time / 60

#     walk_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'walking')
#     data2.at[index, 'walk_time'] = walk_time / 60

# # Print the results
# print(data2)

### Mode specific origin destination travel time for space time prism

In [ ]:
# Function to get trip time for a specific mode
def get_trip_time(origin_lat, origin_long, dest_lat, dest_long, mode_choice, departure_time=None):
    base_url = 'https://dev.virtualearth.net/REST/V1/Routes'
    params = {
        'wp.0': f'{origin_lat},{origin_long}',
        'wp.1': f'{dest_lat},{dest_long}',
        'key': api_key,
    }

    if mode_choice == 'transit':
        base_url += '/Transit'
        if departure_time is not None:
            params['dateTime'] = departure_time
            params['timeType'] = 'Departure'

    elif mode_choice == 'walking':
        base_url += '/Walking'

    else:
        base_url += '/Driving'

    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract the trip duration from the response
    if mode_choice == 'transit':
        trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

    elif mode_choice == 'walking':
        trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']
    
    elif mode_choice=='carpool':
        trip_duration = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['travelDuration']

    else:
        trip_duration = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['travelDuration']

    return trip_duration

In [ ]:
# Iterate through each row and get trip time for each mode
for index, row in data2.iterrows():
    origin_lat = row['o_lat']
    origin_long = row['o_long']
    dest_lat = row['d_lat']
    dest_long = row['d_long']
    mode_choice = row['mode_choice']

    if mode_choice == 'transit':
    # Specify the departure_time for transit mode (replace '2023-07-20T08:00:00' with your desired time)
        departure_time = '2023-07-20T08:00:00'
        transit_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'transit', departure_time)
        travel_time = transit_time / 60
    
    elif mode_choice == 'walking':
        walk_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'walking')
        travel_time = walk_time / 60
    
    elif mode_choice == 'car':
        car_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'car')
        travel_time = car_time / 60
    
    else:
        carpool_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'carpool')
        travel_time = carpool_time * 1.2 / 60
    
    data2.at[index, 'travel_time'] = travel_time

# Print the results
print(data2)